In [1]:
import inspect

In [48]:
def user_required(handler):
    """
         Decorator for checking if there's a user associated with the current session.
         Will also fail if there's no session present.
    """

    def check_login(self, *args, **kwargs):
        if not self.auth.get_user_by_session():
            if isinstance(self, JsonMixin):
                self.write_json_response(self.make_error_response(code=403, msg='not authorized'))
                return
            else:
                self.error(403)
                self.response.write('not authorized')
                return
        else:
            return handler(self, *args, **kwargs)

    return check_login

class HelloHandler(object):
    @user_required
    def get(self, test, *args, **kwargs):
        """Get a greeting endpoint.
        ---
        description: Get a greeting
        responses:
            200:
                description: A greeting to the client
                schema:
                    $ref: '#/definitions/Greeting'
        """
        self.response.write("hello")
    def post(self, test):
        """Get a greeting endpoint.
        ---
        description: Get a greeting
        responses:
            200:
                description: A greeting to the client
                schema:
                    $ref: '#/definitions/Greeting'
        """
        self.response.write("hello")
c = HelloHandler

In [50]:
[k for k, v in inspect.signature(extract_wrapped(getattr(c, 'get'))).parameters.items() if v.kind not in [v.VAR_KEYWORD, v.VAR_POSITIONAL]][1:]

['test']

In [21]:
list(inspect.signature(extract_wrapped(getattr(c, 'get'))).parameters.keys())[1:]

['test', 'args', 'kwargs']

In [11]:
c.get.__closure__[0].__closure__

AttributeError: 'cell' object has no attribute '__closure__'

In [12]:
from types import FunctionType

def extract_wrapped(decorated):
    if decorated is None or decorated.__closure__ is None:
        return decorated
    closure = (c.cell_contents for c in decorated.__closure__)
    return extract_wrapped(next((c for c in closure if isinstance(c, FunctionType)), None))

In [14]:
extract_wrapped(c.post)

<function __main__.HelloHandler.post>